**Post Running the script, restart the runtime in order to run it the other time with some other parameters**

Installing Relevant Packages

In [ ]:
!pip install Scrapy
!pip install crochet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.2/277.2 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 83.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 98.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.0/59.0 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.3/93.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 10.2 MB/s eta 0:00:00


In [ ]:
import scrapy
from scrapy.crawler import CrawlerProcess, CrawlerRunner
# text cleaning
import re
import pandas as pd
from google.colab import auth
import gspread
from google.auth import default
from gspread_dataframe import get_as_dataframe, set_with_dataframe
import gspread_dataframe as gd

Creating a scrapy spider to crawl onto the linkedin page to scrape jobs till it gets the output in the infinite scroll

In the **api_url** below we can add Job Keywords (after **search?keywords=**) and Job Location (after **&location=**) to search for particular type of role

Example:
https://www.linkedin.com/jobs/search?keywords=Developer&location=Mumbai&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0&start=0

This link gives us the developer job role postings in Mumbai.

In [ ]:
class LinkedJobsSpider(scrapy.Spider):
    name = "linkedin_jobs"
    api_url = 'https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Developer&location=Mumbai&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0&start='


    title = []
    url = []
    listed_date = []
    company_name = []
    company_link = []
    company_location = []
    company_logo = []


    def start_requests(self):
        first_job_on_page = 0
        first_url = self.api_url + str(first_job_on_page)
        yield scrapy.Request(url=first_url, callback=self.parse_job, meta={'first_job_on_page': first_job_on_page})


    def parse_job(self, response):
        first_job_on_page = response.meta['first_job_on_page']

        jobs = response.css("li")

        num_jobs_returned = len(jobs)
        print("Num Jobs Returned")
        print(num_jobs_returned)

        for job in jobs:

            self.title.append(job.css("h3::text").get(default='not-found').strip())
            self.url.append(job.css(".base-card__full-link::attr(href)").get(default='not-found').strip())
            self.listed_date.append(job.css('time::attr(datetime)').get(default='not-found').strip())
            self.company_name.append(job.css('h4 a::text').get(default='not-found').strip())
            self.company_link.append(job.css('h4 a::attr(href)').get(default='not-found'))
            self.company_location.append(job.css('.job-search-card__location::text').get(default='not-found').strip())
            self.company_logo.append(job.css('img::attr(data-ghost-url)').get(default='not-found').strip())


        if num_jobs_returned > 0:
          first_job_on_page = int(first_job_on_page) + 25
          next_url = self.api_url + str(first_job_on_page)
          yield scrapy.Request(url=next_url, callback=self.parse_job, meta={'first_job_on_page': first_job_on_page})

Start the crawling process

In [ ]:
process = CrawlerProcess()
process.crawl(LinkedJobsSpider)
process.start()

INFO:scrapy.utils.log:Scrapy 2.9.0 started (bot: scrapybot)
2023-07-15 04:23:27 [scrapy.utils.log] INFO: Scrapy 2.9.0 started (bot: scrapybot)
INFO:scrapy.utils.log:Versions: lxml 4.9.3.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.1, Twisted 22.10.0, Python 3.10.12 (main, Jun  7 2023, 12:45:35) [GCC 9.4.0], pyOpenSSL 23.2.0 (OpenSSL 3.1.1 30 May 2023), cryptography 41.0.2, Platform Linux-5.15.109+-x86_64-with-glibc2.31
2023-07-15 04:23:27 [scrapy.utils.log] INFO: Versions: lxml 4.9.3.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.1, Twisted 22.10.0, Python 3.10.12 (main, Jun  7 2023, 12:45:35) [GCC 9.4.0], pyOpenSSL 23.2.0 (OpenSSL 3.1.1 30 May 2023), cryptography 41.0.2, Platform Linux-5.15.109+-x86_64-with-glibc2.31
INFO:scrapy.crawler:Overridden settings:
{}
2023-07-15 04:23:27 [scrapy.crawler] INFO: Overridden settings:
{}
/usr/local/lib/python3.10/dist-packages/scrapy/utils/request.py:232: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'R

Num Jobs Returned
25


DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Developer&location=Mumbai&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0&start=25> (referer: https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Developer&location=Mumbai&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0&start=0)
2023-07-15 04:23:29 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Developer&location=Mumbai&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0&start=25> (referer: https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Developer&location=Mumbai&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0&start=0)


Num Jobs Returned
25


DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Developer&location=Mumbai&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0&start=50> (referer: https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Developer&location=Mumbai&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0&start=25)
2023-07-15 04:23:29 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Developer&location=Mumbai&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0&start=50> (referer: https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Developer&location=Mumbai&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0&start=25)


...

DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Developer&location=Mumbai&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0&start=200> (referer: https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Developer&location=Mumbai&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0&start=175)
2023-07-15 04:23:33 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Developer&location=Mumbai&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0&start=200> (referer: https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Developer&location=Mumbai&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0&start=175)


Num Jobs Returned
25


DEBUG:scrapy.core.engine:Crawled (200) <GET https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Developer&location=Mumbai&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0&start=225> (referer: https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Developer&location=Mumbai&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0&start=200)
2023-07-15 04:23:34 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Developer&location=Mumbai&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0&start=225> (referer: https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Developer&location=Mumbai&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0&start=200)
DEBUG:scrapy.downloadermiddlewares.retry:Retrying <GET https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keyword

Num Jobs Returned
25


DEBUG:scrapy.downloadermiddlewares.retry:Retrying <GET https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Developer&location=Mumbai&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0&start=250> (failed 2 times): 429 Unknown Status
2023-07-15 04:23:34 [scrapy.downloadermiddlewares.retry] DEBUG: Retrying <GET https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Developer&location=Mumbai&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0&start=250> (failed 2 times): 429 Unknown Status
ERROR:scrapy.downloadermiddlewares.retry:Gave up retrying <GET https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Developer&location=Mumbai&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0&start=250> (failed 3 times): 429 Unknown Status
2023-07-15 04:23:34 [scrapy.downloadermiddlewares.retry] ERROR: Gave up retrying <GET https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJob

Printing the output for one of the crawled columns: Company_Name

In [ ]:
for data in LinkedJobsSpider.company_name:
  print(data)

Rejolut- A Web 3.0 Company
StatusNeo
SUGAR Cosmetics
Acme Services
Mosaic Wellness
NeoSOFT
Gupshup
SUGAR Cosmetics
SlashRTC
Coconut Media Box
GST Edge
Acme Services
Peachmode.com
Getsethome
VLaunchU
Chalo
Qrata
Antal International
Unotech Software Pvt. Ltd
Acme Services
Rejolut- A Web 3.0 Company
DigiThinkIt Inc.
Wissen Technology
Apprend Innovations Pvt. Ltd.
Horizon Financial & Properties Advisers Pvt Ltd
Thyrocare Technologies Ltd.
BookMyShow
TAC Security
Altorise
Tessact
Aurum PropTech Limited
Chien Technologies Pvt Ltd
Apollo Finvest
SUGAR Cosmetics
Aurum PropTech Limited
SySpree Digital
WEQ Technologies
Gupshup
Falah Technologies
Acme Services
not-found
Gupshup
edge CRM - Your Strategic Growth Partner.
Bit Brothers
Acme Services
SUGAR Cosmetics
Qrata
FactWise
Dotcom Services (India) Pvt. Ltd.
i2e Consulting
Cogno AI
Rejolut- A Web 3.0 Company
Acme Services
Acme Services
Jai Kisan
Talent Corner HR Services Pvt Ltd
JSW One Platforms Ltd.
Acme Services
Bee Online Communication Pvt. 

Creating a dataframe to store the scraped data

In [ ]:
data = pd.DataFrame(columns=['job_title','job_detail_url','job_listed','company_name','company_link','company_location', 'company_logo'])

Filling the data in the datafram

In [ ]:
data['job_title'] = LinkedJobsSpider.title
data['job_detail_url'] = LinkedJobsSpider.url
data['job_listed'] = LinkedJobsSpider.listed_date
data['company_name'] = LinkedJobsSpider.company_name
data['company_link'] = LinkedJobsSpider.company_link
data['company_location'] = LinkedJobsSpider.company_location
data['company_logo'] = LinkedJobsSpider.company_logo

In [ ]:
data.head()

,job_title,job_detail_url,job_listed,company_name,company_link,company_location,company_logo
0,HTML - CSS Developer,https://in.linkedin.com/jobs/view/html-css-dev...,2023-06-12,Rejolut- A Web 3.0 Company,https://in.linkedin.com/company/rejolut?trk=pu...,"Mumbai, Maharashtra, India",https://static.licdn.com/aero-v1/sc/h/9a9u41th...
1,Frontend Developers,https://in.linkedin.com/jobs/view/frontend-dev...,2023-06-06,StatusNeo,https://www.linkedin.com/company/statusneo?trk...,"Mumbai, Maharashtra, India",https://static.licdn.com/aero-v1/sc/h/9a9u41th...
2,Frontend Developer,https://in.linkedin.com/jobs/view/frontend-dev...,2023-04-10,SUGAR Cosmetics,https://in.linkedin.com/company/sugar-cosmetic...,"Mumbai, Maharashtra, India",https://static.licdn.com/aero-v1/sc/h/9a9u41th...
3,HTML/CSS Developer,https://in.linkedin.com/jobs/view/html-css-dev...,2023-06-16,Acme Services,https://in.linkedin.com/company/acmeservices?t...,"Mumbai, Maharashtra, India",https://static.licdn.com/aero-v1/sc/h/9a9u41th...
4,Jr Frontend Developer,https://in.linkedin.com/jobs/view/jr-frontend-...,2023-06-06,Mosaic Wellness,https://in.linkedin.com/company/mosaicwellness...,"Mumbai, Maharashtra, India",https://static.licdn.com/aero-v1/sc/h/9a9u41th...


Opening the Google Sheet where we want to write the data

In [ ]:
auth.authenticate_user()
creds, _ = default()

gc = gspread.authorize(creds)

worksheet = gc.open('Jobs_Data').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

DEBUG:google.auth.transport._http_client:Making request: GET http://169.254.169.254
2023-07-15 04:24:23 [google.auth.transport._http_client] DEBUG: Making request: GET http://169.254.169.254
DEBUG:google.auth.transport._http_client:Making request: GET http://metadata.google.internal/computeMetadata/v1/project/project-id
2023-07-15 04:24:23 [google.auth.transport._http_client] DEBUG: Making request: GET http://metadata.google.internal/computeMetadata/v1/project/project-id
DEBUG:google.auth.transport.requests:Making request: GET http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/default/?recursive=true
2023-07-15 04:24:23 [google.auth.transport.requests] DEBUG: Making request: GET http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/default/?recursive=true
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): metadata.google.internal:80
2023-07-15 04:24:23 [urllib3.connectionpool] DEBUG: Starting new HTTP connection (1): metadata

In [ ]:
print(rows)

[['job_title', 'job_detail_url', 'job_listed', 'company_name', 'company_link', 'company_location', 'company_logo'], ['Probationary Officer', 'not-found', '2023-06-16', 'not-found', 'not-found', 'Bengaluru, Karnataka, India', 'https://static.licdn.com/aero-v1/sc/h/9a9u41thxt325ucfh5z8ga4m8'], ['Wildcraft - Product/Category Manager', 'https://in.linkedin.com/jobs/view/wildcraft-product-category-manager-at-wildcraft-india-3656723968?refId=328qeewAZCTCR%2FKwkJKjVg%3D%3D&trackingId=XFmheGYcpNEJW9bwGqOxzQ%3D%3D&position=2&pageNum=0&trk=public_jobs_jserp-result_search-card', '2023-07-07', 'Wildcraft India', 'https://in.linkedin.com/company/wildcraft?trk=public_jobs_jserp-result_job-search-card-subtitle', 'Bengaluru, Karnataka, India', 'https://static.licdn.com/aero-v1/sc/h/9a9u41thxt325ucfh5z8ga4m8'], ['Swiggy - Regional Gourmet Manager', 'https://in.linkedin.com/jobs/view/swiggy-regional-gourmet-manager-at-swiggy-3656725129?refId=328qeewAZCTCR%2FKwkJKjVg%3D%3D&trackingId=iT9Oimf7tb7bhce%2F3Q

## If writing the data for the first time in the sheet - use the below code **otherwise do not run this** You can check that from the above row - if the above row gives no data, the sheet is empty otherwise not

In [ ]:
worksheet.clear()
set_with_dataframe(worksheet=worksheet, dataframe=data, include_index=False, include_column_header=True, resize=True)

DEBUG:urllib3.connectionpool:https://sheets.googleapis.com:443 "POST /v4/spreadsheets/1rKcin7kL-WT_8V7kKfzTk_n1Peo-99erjp4n1Xsw93c/values/%27Sheet1%27:clear HTTP/1.1" 200 None
2023-07-15 02:23:17 [urllib3.connectionpool] DEBUG: https://sheets.googleapis.com:443 "POST /v4/spreadsheets/1rKcin7kL-WT_8V7kKfzTk_n1Peo-99erjp4n1Xsw93c/values/%27Sheet1%27:clear HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://sheets.googleapis.com:443 "POST /v4/spreadsheets/1rKcin7kL-WT_8V7kKfzTk_n1Peo-99erjp4n1Xsw93c:batchUpdate HTTP/1.1" 200 None
2023-07-15 02:23:18 [urllib3.connectionpool] DEBUG: https://sheets.googleapis.com:443 "POST /v4/spreadsheets/1rKcin7kL-WT_8V7kKfzTk_n1Peo-99erjp4n1Xsw93c:batchUpdate HTTP/1.1" 200 None
DEBUG:gspread_dataframe:1932 cell updates to send
2023-07-15 02:23:18 [gspread_dataframe] DEBUG: 1932 cell updates to send
DEBUG:urllib3.connectionpool:https://sheets.googleapis.com:443 "PUT /v4/spreadsheets/1rKcin7kL-WT_8V7kKfzTk_n1Peo-99erjp4n1Xsw93c/values/%27Sheet1%27%21A1%

We can go and check the sheet if the data has been added into the sheet

# If we are trying to append the data into an already filled sheet - **use the below code**

In [ ]:
existing = gd.get_as_dataframe(worksheet)
updated = existing.append(data)
gd.set_with_dataframe(worksheet, updated)

DEBUG:urllib3.connectionpool:https://sheets.googleapis.com:443 "GET /v4/spreadsheets/1rKcin7kL-WT_8V7kKfzTk_n1Peo-99erjp4n1Xsw93c/values/Sheet1?valueRenderOption=FORMULA&dateTimeRenderOption=FORMATTED_STRING HTTP/1.1" 200 None
2023-07-15 04:25:56 [urllib3.connectionpool] DEBUG: https://sheets.googleapis.com:443 "GET /v4/spreadsheets/1rKcin7kL-WT_8V7kKfzTk_n1Peo-99erjp4n1Xsw93c/values/Sheet1?valueRenderOption=FORMULA&dateTimeRenderOption=FORMATTED_STRING HTTP/1.1" 200 None
<ipython-input-12-10ae62e10f0d>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  updated = existing.append(data)
DEBUG:urllib3.connectionpool:https://sheets.googleapis.com:443 "POST /v4/spreadsheets/1rKcin7kL-WT_8V7kKfzTk_n1Peo-99erjp4n1Xsw93c:batchUpdate HTTP/1.1" 200 None
2023-07-15 04:25:56 [urllib3.connectionpool] DEBUG: https://sheets.googleapis.com:443 "POST /v4/spreadsheets/1rKcin7kL-WT_8V7kKfzTk_n1Peo-99erjp4n1Xsw93c:batch

Sheet Link: https://docs.google.com/spreadsheets/d/1rKcin7kL-WT_8V7kKfzTk_n1Peo-99erjp4n1Xsw93c/edit?usp=sharing

1. Using Keywords, Location and Other Filters to maximize the list of companies scraped.
2. Creating Daily Jobs to run in order to scrap new postings daily